# Introduction

This tutorial covers submission scoring and conversion scripts. 

Our starter kit includes the following files:
* `parsers.py` contains all the code for parsing the XML-like intermediate representations and annotated examples into our `ProblemFormulation` dataclass. 
* `scoring.py` contains the code to score your submission. The functions here take in the canonical representations of the problem formulations.


# Dataclasses

The primary dataclass we use is `ProblemFormulation`. 

```
class ProblemFormulation:
    objective: ObjectiveDeclaration
    constraints: list[ConstraintDeclaration]
    entities: dict[str, int]
```
It has an objective, a list of constraints, and a dictionary of entities which maps each problem entity to an index. This index is used as its index in the canonical representation.

`ProblemFormulation` serves as an intermediate representation for participants that can be reused by participants if desired. However, the canonical representation is what will be used for scoring and serves as the unique format in evaluating submissions. To convert from `ProblemFormulation` to `CanonicalFormulation`, we also include a `convert_to_canonical:ProblemFormulation -> CanonicalFormulation` function if you choose to use our dataclasses.


![title](demo.drawio.svg)


# Canonical Representation

As a brief overview. The canonical representation in our case describes the objectives and constraints as a vector and matrix respectively.

For the objective declaration, we may have an algebraic formulation of the form

$$ \max 0.007x_0 + 0.002x_1. $$

This corresponds to a canonical objective vector as shown:

$$ \begin{bmatrix} 0.007 & 0.002\end{bmatrix}^T. $$

For the constraints, we may have a set of constraints of the form

$$ \begin{align*}
x_0 + x_1 &\le 60000\\
x_0 &\ge 0.015(x_0+x_1)\\
x_1 & \le 0.6(x_0+x_1)
\end{align*}.$$

We aim to have this in the form $Ax \le b$, and our canonical form would be the matrix $\begin{bmatrix} A & b \end{bmatrix}$. We start by moving all constant terms to the right side and all variables to the left. Then we flip inequalities to match:

$$ \begin{align*}
x_0 + x_1 &\le 60000\\
-0.085 x_0 + 0.015 x_1 &\le 0\\
-0.6 x_0 + 0.4 x_1 &\le 0
\end{align*}.$$

Then our canonical form for the constraints would be:

$$ \begin{bmatrix}
1 & 1 & 60000\\
-0.085 & 0.015 & 0\\
-0.6 & 0.4 & 0
\end{bmatrix}. $$


## Order Mapping

Note that the order of the columns matter! We will give an `order_mapping` dictionary in the ground truth to ensure consistency in the column indicies. For example, our order mapping may look something like:
```
{
    'apples': 0,
    'oranges': 1,
    'grapes': 2
}
```
This would mean apples would be in the first column of our canonical form matrices, oranges in the second, and grapes in the third. If you are using a self-written parser, ensure that your parser takes in this order mapping parameter and properly maps it to the correct column. 

# Parsing 

You may skip this if you intend to preprocess your model outputs yourself into the canonical output.

In [1]:
import sys
sys.path.append("..")

In [2]:
import parsers
import jsonlines
import prettyprinter as pp

pp.install_extras(exclude=['python', 'django'])

## XML Parsing

Here, we will go through parsing the XML output that the baseline model produces.

In [3]:
# read original file 
# this is an example of the intermediate output
with open('demo_examples/modeloutput1.txt', 'r') as fd:
    data = fd.read()
    print(data)

<DECLARATION>
<OBJ_DIR> maximize </OBJ_DIR>
<OBJ_NAME> revenue </OBJ_NAME> [is]
<VAR> turnips </VAR> [TIMES] <PARAM> 300 </PARAM>
<VAR> pumpkins </VAR> [TIMES] <PARAM> 450 </PARAM>
</DECLARATION>
<DECLARATION>
<CONST_DIR> has </CONST_DIR><LIMIT> 500 </LIMIT>
<CONST_TYPE> [SUM_CONSTRAINT] </CONST_TYPE>
</DECLARATION>
<DECLARATION>
<CONST_DIR> available </CONST_DIR>
<OPERATOR> LESS_OR_EQUAL </OPERATOR>
<LIMIT> 40000</LIMIT>
<CONST_TYPE> [LINEAR_CONSTRAINT] </CONST_TYPE> [is]
<VAR> Turnips </VAR> [TIMES] <PARAM> 50 </PARAM>
<VAR> Pumpkins </VAR> [TIMES] <PARAM> 90 </PARAM>
</DECLARATION>
<DECLARATION>
<CONST_DIR> available </CONST_DIR>
<OPERATOR> LESS_OR_EQUAL </OPERATOR>
<LIMIT> 34000 </LIMIT>
<CONST_TYPE> [LINEAR_CONSTRAINT] </CONST_TYPE> [is]
<VAR> Turnips </VAR> [TIMES] <PARAM> 80 </PARAM>
<VAR> Pumpkins </VAR> [TIMES] <PARAM> 40 </PARAM>
</DECLARATION>


In [4]:
# instantiate parser
parser = parsers.ModelOutputXMLParser()

# parse our file
# parser.read_xml will read a string already in memory
formulation = parser.parse_file('demo_examples/modeloutput1.txt')

pp.pprint(formulation)

parsers.ProblemFormulation(
    objective=parsers.ObjectiveDeclaration(
        direction='maximize',
        terms={
            'turnips': parsers.Term(name='turnips', index=0, value=300.0),
            'pumpkins': parsers.Term(name='pumpkins', index=1, value=450.0)
        },
        entities={'turnips': 0, 'pumpkins': 1},
        name='revenue'
    ),
    constraints=[
        parsers.ConstraintDeclaration(
            direction='has',
            terms=collections.OrderedDict([]),
            entities={'turnips': 0, 'pumpkins': 1},
            type='[SUM_CONSTRAINT]',
            limit=500.0,
            operator=''
        ),
        parsers.ConstraintDeclaration(
            direction='available',
            terms=collections.OrderedDict([
                ('turnips', parsers.Term(name='turnips', index=0, value=50.0)),
                (
                    'pumpkins',
                    parsers.Term(name='pumpkins', index=1, value=90.0)
                )
            ]),
       

In [5]:
# convert to canonical form
canonical = parsers.convert_to_canonical(formulation)
print(f'Objective:\n {canonical.objective}')
print(f'Constraints:\n {canonical.constraints}')

Objective:
 [300. 450.]
Constraints:
 [[1.0e+00 1.0e+00 5.0e+02]
 [5.0e+01 9.0e+01 4.0e+04]
 [8.0e+01 4.0e+01 3.4e+04]]


We use `xml.etree.ElementTree` as our XML parser. If your model outputs are XML, **please post-process your outputs and ensure they are syntactically correct**. We make some best-effort attempts at parsing syntactically incorrect outputs using the BeautifulSoup package, but this does not guarantee that an syntactically incorrect output gets parsed correctly.

## JSON Formulation Parsing

Here, we go through parsing the JSON formatted formulations in the ground truths.

In [6]:
parser = parsers.JSONFormulationParser()
# read JSON file
with jsonlines.open('demo_examples/annotated.jsonl') as reader:
    unparsed = [line for line in reader.iter()]
    pp.pprint(unparsed[0])

{
    '-640645082': {
        'document':
            'A coconut seller has to transport coconuts using either rickshaws or '
            'ox carts. The rickshaws can take 50 coconuts each and cost $10 per '
            'trip. The ox carts can take 30 coconuts each and cost $8 per trip. '
            'The seller has at most $200 to spend on transporting the coconuts. '
            'Due to pollution, the number of rickshaws must not exceed the number '
            'of ox carts. Formulate a LP to maximize the number of coconuts that '
            'can be transported.',
        'vars': ['rickshaws', 'ox carts'],
        'var_mentions': [
            'rickshaws',
            'ox carts',
            'rickshaws',
            'ox carts',
            'rickshaws',
            'ox carts'
        ],
        'params': ['50', '10', '30', '8'],
        'var_mention_to_first_var': {
            'rickshaws': 'rickshaws',
            'ox carts': 'ox carts'
        },
        'first_var_to_mentions': {


Parsing this data, we get the `ProblemFormulation` representation again.

In [7]:
# parse JSON representation
parsed = [parser.parse(line) for line in unparsed]
pp.pprint(parsed[0])

parsers.ProblemFormulation(
    objective=parsers.ObjectiveDeclaration(
        direction='maximize',
        terms={
            'rickshaws': parsers.Term(name='rickshaws', index=0, value=50.0),
            'ox carts': parsers.Term(name='ox carts', index=1, value=30.0)
        },
        entities={'rickshaws': 0, 'ox carts': 1},
        name='number of coconuts'
    ),
    constraints=[
        parsers.ConstraintDeclaration(
            direction='at most',
            terms=collections.OrderedDict([
                (
                    'rickshaws',
                    parsers.Term(name='rickshaws', index=0, value=10.0)
                ),
                ('ox carts', parsers.Term(name='ox carts', index=1, value=8.0))
            ]),
            entities={'rickshaws': 0, 'ox carts': 1},
            type='[LINEAR_CONSTRAINT]',
            limit=200.0,
            operator='LESS_OR_EQUAL'
        ),
        parsers.ConstraintDeclaration(
            direction='must not exceed',
        

# Scoring

When scoring, your model outputs will be compared to the canonical form of the annotated datasest. We have provided in `demo_examples` some examples:
* `modeloutput*.txt` are two XML intermediate outputs for two problem formulations.
* `annotated.jsonl` contains two annotated JSON examples of the same problem formulations.

Suppose that for this following example, your model outputs XML outputs. You must first parse them then convert them to canonical form. If your model outputs the canonical form, you may directly use the canonical form directly without parsing.

In [8]:
json_parser = parsers.JSONFormulationParser()
xml_parser = parsers.ModelOutputXMLParser()
# regular test pipeline of a few examples
with jsonlines.open('demo_examples/annotated.jsonl') as reader:
    true_formulations = []
    predicted = []

    for i, line in enumerate(reader.iter()):
        true_formulation = json_parser.parse(line)
        true_formulations.append(true_formulation)
        # make sure you use the order mapping from the ground truth (true_formulation.entities)
        predicted.append(xml_parser.parse_file(f'demo_examples/modeloutput{i}.txt', true_formulation.entities))

In [9]:
pp.pprint(predicted[1])

parsers.ProblemFormulation(
    objective=parsers.ObjectiveDeclaration(
        direction='maximize',
        terms={
            'turnips': parsers.Term(name='turnips', index=0, value=300.0),
            'pumpkins': parsers.Term(name='pumpkins', index=1, value=450.0)
        },
        entities={
            'turnips': 0,
            'pumpkins': 1,
            'Turnips': 0,
            'Pumpkins': 1
        },
        name='turnips'
    ),
    constraints=[
        parsers.ConstraintDeclaration(
            direction='has',
            terms=collections.OrderedDict([]),
            entities={
                'turnips': 0,
                'pumpkins': 1,
                'Turnips': 0,
                'Pumpkins': 1
            },
            type='[SUM_CONSTRAINT]',
            limit=500.0,
            operator=''
        ),
        parsers.ConstraintDeclaration(
            direction='available',
            terms=collections.OrderedDict([
                ('Turnips', parsers.Term(name='T

In [10]:
pp.pprint(true_formulations[1])

parsers.ProblemFormulation(
    objective=parsers.ObjectiveDeclaration(
        direction='maximize',
        terms={
            'turnips': parsers.Term(name='turnips', index=0, value=300.0),
            'pumpkins': parsers.Term(name='pumpkins', index=1, value=450.0)
        },
        entities={
            'turnips': 0,
            'pumpkins': 1,
            'Turnips': 0,
            'Pumpkins': 1
        },
        name='revenue'
    ),
    constraints=[
        parsers.ConstraintDeclaration(
            direction='has',
            terms=collections.OrderedDict([]),
            entities={
                'turnips': 0,
                'pumpkins': 1,
                'Turnips': 0,
                'Pumpkins': 1
            },
            type='[SUM_CONSTRAINT]',
            limit=500.0,
            operator='LESS_OR_EQUAL'
        ),
        parsers.ConstraintDeclaration(
            direction='available',
            terms=collections.OrderedDict([
                ('Turnips', parsers

Notice that in the third constraint, `pumpkins` is different in the predicted (40) compared to the ground truth (50). This will count as a false positive. To illustrate as the canonical form constraints:

In [11]:
p1 = parsers.convert_to_canonical(predicted[1])
t1 = parsers.convert_to_canonical(true_formulations[1])

print(p1.constraints)
print('\n')
print(t1.constraints)
# notice that the last row is different

[[1.0e+00 1.0e+00 1.0e+00 1.0e+00 5.0e+02]
 [5.0e+01 9.0e+01 1.0e+00 1.0e+00 4.0e+04]
 [8.0e+01 4.0e+01 1.0e+00 1.0e+00 3.4e+04]]


[[1.0e+00 1.0e+00 1.0e+00 1.0e+00 5.0e+02]
 [5.0e+01 9.0e+01 1.0e+00 1.0e+00 4.0e+04]
 [8.0e+01 5.0e+01 1.0e+00 1.0e+00 3.4e+04]]


We compute the false positives (FP), false negatives (FN), and number of ground-truth declarations (D) for each example.
* FP is the number of non-matched predicted declarations. For example, an extra predicted constraint or an incorrect value in a constraint.
* FN denotes the number of ground-truth declarations without a match. For example, too few predicted constraints.
* D is the number of ground-truth declarations. This is equal to the number of constraints + the number of objectives (1) in the ground truth.

To get FP, FN, D on one example:

In [12]:
import scoring

fp, fn, d = scoring.per_example_scores(p1.objective, p1.constraints, t1.objective, t1.constraints)
print(f'FP: {fp}')
print(f'FN: {fn}')
print(f'D: {d}')

FP: 1
FN: 0
D: 4


Then to get the overall score on all examples, we pass in the lists of objectives and constraints. These all should be given as `list[np.ndarray]`.

In [13]:
pred_canonical = [parsers.convert_to_canonical(f) for f in predicted]
true_canonical = [parsers.convert_to_canonical(f) for f in true_formulations]

# get objectives and constraints
pred_obj = [f.objective for f in pred_canonical]
pred_const = [f.constraints for f in pred_canonical]
true_obj = [f.objective for f in true_canonical]
true_const = [f.constraints for f in true_canonical]

score = scoring.overall_score(pred_obj, pred_const, true_obj, true_const)

print(score)

0.8571428571428572
